In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [40]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Dropout,Dense

In [4]:
df = pd.read_csv("./creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df["Time"] = df["Time"].apply(lambda x: (x/3600)%24)

In [6]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.000000,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.000000,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,0.000278,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,0.000278,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,0.000556,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [14]:
normal_trans = df[df["Class"] == 0].sample(1000)
fraud_trans = df[df["Class"] == 1]
reduced_trans = pd.concat([normal_trans,fraud_trans]).reset_index(drop = True)
print(reduced_trans.shape)

(1492, 31)


In [19]:
features = reduced_trans.drop("Class",axis=1)
target = reduced_trans["Class"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

train_index = y_train[y_train == 1].index
x_train = X_train.loc[train_index]
print(x_train.shape)

(399, 30)


In [21]:
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(X_test)

In [43]:
class AutoEncoder(Model):
    def __init__(self,output_dim,ldim=8):
        super().__init__()
        self.Encoder = keras.Sequential([
            keras.layers.Dense(64, activation="relu"),
            Dropout(0.1),
            keras.layers.Dense(32, activation="relu"),
            Dropout(0.1),
            keras.layers.Dense(16, activation="relu"),
            Dropout(0.1),
            keras.layers.Dense(ldim, activation="relu"),
        ])
        self.Decoder = keras.Sequential([
            keras.layers.Dense(16, activation="relu"),
            Dropout(0.1),
            keras.layers.Dense(32, activation="relu"),
            Dropout(0.1),
            keras.layers.Dense(64, activation="relu"),
            Dropout(0.1),
            keras.layers.Dense(output_dim, activation="sigmoid"),
        ])
    
    def call(self,input):
        encoded = self.Encoder(input)
        decoded = self.Decoder(encoded)
        return decoded

In [44]:
model=AutoEncoder(output_dim=x_train_scaled.shape[1])
model.compile(optimizer="adam",loss="mse")

history = model.fit(
    x_train_scaled, x_train_scaled,
    epochs=20, batch_size=512,
    validation_data=(x_test_scaled, x_test_scaled),
)

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.0577 - val_loss: 0.0747
Epoch 2/20
1/1 [==============================] - 0s 74ms/step - loss: 0.0574 - val_loss: 0.0743
Epoch 3/20
1/1 [==============================] - 0s 74ms/step - loss: 0.0571 - val_loss: 0.0740
Epoch 4/20
1/1 [==============================] - 0s 72ms/step - loss: 0.0568 - val_loss: 0.0737
Epoch 5/20
1/1 [==============================] - 0s 72ms/step - loss: 0.0566 - val_loss: 0.0733
Epoch 6/20
1/1 [==============================] - 0s 72ms/step - loss: 0.0563 - val_loss: 0.0729
Epoch 7/20
1/1 [==============================] - 0s 64ms/step - loss: 0.0560 - val_loss: 0.0725
Epoch 8/20
1/1 [==============================] - 0s 73ms/step - loss: 0.0556 - val_loss: 0.0720
Epoch 9/20
1/1 [==============================] - 0s 73ms/step - loss: 0.0553 - val_loss: 0.0714
Epoch 10/20
1/1 [==============================] - 0s 70ms/step - loss: 0.0548 - val_loss: 0.0707
Epoch 11/20
1/1 [==============